Mod 26

In [2]:
def decrypt_rot13(text):
    decrypted_text = ""
    for char in text:
        if 'A' <= char <= 'Z':
            decrypted_char = chr(((ord(char) - ord('A') - 13) % 26) + ord('A'))
        elif 'a' <= char <= 'z':
            decrypted_char = chr(((ord(char) - ord('a') - 13) % 26) + ord('a'))
        else:
            decrypted_char = char
        decrypted_text += decrypted_char
    return decrypted_text

rot13_encoded_string = "cvpbPGS{arkg_gvzr_V'yy_gel_2_ebhaqf_bs_ebg13_jdJBFOXJ}"
decrypted_string = decrypt_rot13(rot13_encoded_string)
print(decrypted_string)

picoCTF{next_time_I'll_try_2_rounds_of_rot13_wqWOSBKW}


Easy Peasy

In [21]:
# Code obtained from writeup (I didn't understand so I looked it up)
# Code found at: https://github.com/Dvd848/CTFs/blob/master/2021_picoCTF/Easy_Peasy.md
from pwn import *

KEY_LEN = 50000
MAX_CHUNK = 1000

r = remote("mercury.picoctf.net", 41934)
r.recvuntil("This is the encrypted flag!\n")
flag = r.recvlineS(keepends = False)
log.info(f"Flag: {flag}")
bin_flag = unhex(flag)

counter = KEY_LEN - len(bin_flag)

with log.progress('Causing wrap-around') as p:
    while counter > 0:
        p.status(f"{counter} bytes left")
        chunk_size = min(MAX_CHUNK, counter)
        r.sendlineafter("What data would you like to encrypt? ", "a" * chunk_size)
        
        counter -= chunk_size

r.sendlineafter("What data would you like to encrypt? ", bin_flag)
r.recvlineS()
log.success("The flag: {}".format(unhex(r.recvlineS())))


[x] Opening connection to mercury.picoctf.net on port 41934
[x] Opening connection to mercury.picoctf.net on port 41934: Trying 18.189.209.142
[+] Opening connection to mercury.picoctf.net on port 41934: Done


/var/folders/9q/ndfgzc_15fjcl7lb_ynn97640000gn/T/ipykernel_88077/931727646.py:7: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.recvuntil("This is the encrypted flag!\n")


[*] Flag: 0345376e1e5406691d5c076c4050046e4000036a1a005c6b1904531d3941055d
[x] Causing wrap-around
[x] Causing wrap-around: 49968 bytes left
[x] Causing wrap-around: 48968 bytes left


/var/folders/9q/ndfgzc_15fjcl7lb_ynn97640000gn/T/ipykernel_88077/931727646.py:18: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendlineafter("What data would you like to encrypt? ", "a" * chunk_size)


[x] Causing wrap-around: 47968 bytes left
[x] Causing wrap-around: 46968 bytes left
[x] Causing wrap-around: 45968 bytes left
[x] Causing wrap-around: 44968 bytes left
[x] Causing wrap-around: 43968 bytes left
[x] Causing wrap-around: 42968 bytes left
[x] Causing wrap-around: 41968 bytes left
[x] Causing wrap-around: 40968 bytes left
[x] Causing wrap-around: 39968 bytes left
[x] Causing wrap-around: 38968 bytes left
[x] Causing wrap-around: 37968 bytes left
[x] Causing wrap-around: 36968 bytes left
[x] Causing wrap-around: 35968 bytes left
[x] Causing wrap-around: 34968 bytes left
[x] Causing wrap-around: 33968 bytes left
[x] Causing wrap-around: 32968 bytes left
[x] Causing wrap-around: 31968 bytes left
[x] Causing wrap-around: 30968 bytes left
[x] Causing wrap-around: 29968 bytes left
[x] Causing wrap-around: 28968 bytes left
[x] Causing wrap-around: 27968 bytes left
[x] Causing wrap-around: 26968 bytes left
[x] Causing wrap-around: 25968 bytes left
[x] Causing wrap-around: 24968 byt

The Numbers

In [3]:
def decrypt_index_cipher(ciphertext):
    decrypted_text = ""
    i = 0

    while i < len(ciphertext):
        if ciphertext[i].isdigit():
            # Find the end of the number
            j = i
            while j < len(ciphertext) and ciphertext[j].isdigit():
                j += 1

            # Get the numeric part and convert it to an integer
            index = int(ciphertext[i:j])

            # Convert the index to the corresponding letter (A=1, B=2, ..., Z=26)
            if 1 <= index <= 26:
                decrypted_char = chr(ord('A') + index - 1)
                decrypted_text += decrypted_char
            else:
                # Handle invalid indices
                decrypted_text += "[INVALID]"
            
            # Move the index to the next non-digit character
            i = j
        else:
            # Copy non-digit characters unchanged
            decrypted_text += ciphertext[i]
            i += 1

    return decrypted_text

# Example usage:
ciphertext1 = "16 9 3 15 3 20 6"
ciphertext2 = "20 8 5 14 21 13 2 5 18 19 13 1 19 15 14"
decrypted_string_out = decrypt_index_cipher(ciphertext1)
decrypted_string_in = decrypt_index_cipher(ciphertext2)

print(decrypted_string_out)
print(decrypted_string_in)


P I C O C T F
T H E N U M B E R S M A S O N


New Caesar

In [6]:
import string

LOWERCASE_OFFSET = ord("a")
ALPHABET = string.ascii_lowercase[:16]

def b16_encode(plain):
    enc = ""
    for c in plain:
        binary = "{0:08b}".format(ord(c))
        enc += ALPHABET[int(binary[:4], 2)]
        enc += ALPHABET[int(binary[4:], 2)]
    return enc
def b16_decode(ciphered):
    decoded=""
    for i in range(0,len(ciphered),2):
        binary="{0:04b}".format(ALPHABET.index(ciphered[i]))+"{0:04b}".format(ALPHABET.index(ciphered[i+1]))
        decoded+=chr(int(binary,2))
    return decoded

def shift(c, k):
    t1 = ord(c) - LOWERCASE_OFFSET
    t2 = ord(k) - LOWERCASE_OFFSET
    return ALPHABET[(t1 + t2) % len(ALPHABET)]

flag = "ihjghbjgjhfbhbfcfjflfjiifdfgffihfeigidfligigffihfjfhfhfhigfjfffjfeihihfdieieih"

for key in ALPHABET:
    dec=""
    for c in flag:
        dec+=shift(c,key)
    b16=b16_decode(dec)
    print(key,b16)

a qQqRY[YSVUT[UYWWWYUYTS
b §§¨bcjljdgfelfjhhhjfjed
c ©¸¸¹st{}{ªuxw©v¨¥}¨¨w©{yyy¨{w{v©©u¦¦©
d ºÉ¤ÉÊ¤»º¹¶¹¹º¹ºº··º
e ËÚµÚÛµÌËÊÇÊÊËÊËËÈÈË
f ÜëÆëì¦Æ§® ®Ý¨«ªÜ©ÛØ ÛÛªÜ®¬¬¬Û®ª®©ÜÜ¨ÙÙÜ
g íü×üý·×¸¿±¿î¹¼»íºìé±ìì»í¿½½½ì¿»¿ºíí¹êêí
ÈèÉÀÂÀÿÊÍÌþËýúÂýýÌþÀÎÎÎýÀÌÀËþþÊûûþ
i ùÙùÚÑÓÑ ÛÞÝÜÓÝÑßßßÑÝÑÜÛ
j /
/ ê
ëâäâìïîíäîâàààâîâíì
k !001ûüóõó"ýðÿ!þ -õ  ÿ!óñññ óÿóþ!!ý..!
3 21>11 21 22??2
m CR=RS=DCBOBBCBCC@@C
n TcNcd.N/&(&U #"T!SP(SS"T&$$$S&"&!TT QQT
o et_tu?_0797f143e2da9dd3e7555d7372ee1bbe
p v`@`AHJHwBEDvCurJuuDvHFFFuHDHCvvBssv


13

In [7]:
def decode_rot13(text):
    decoded_text = ""
    for char in text:
        if 'A' <= char <= 'Z':
            # Decode uppercase letters
            decoded_char = chr(((ord(char) - ord('A') + 13) % 26) + ord('A'))
        elif 'a' <= char <= 'z':
            # Decode lowercase letters
            decoded_char = chr(((ord(char) - ord('a') + 13) % 26) + ord('a'))
        else:
            # Leave non-alphabetical characters unchanged
            decoded_char = char
        decoded_text += decoded_char
    return decoded_text

# Example usage:
rot13_encoded_string = "cvpbPGS{abg_gbb_onq_bs_n_ceboyrz}"
decoded_string = decode_rot13(rot13_encoded_string)
print(decoded_string)


picoCTF{not_too_bad_of_a_problem}


Caesar

In [19]:
def decode_caesar_cipher(ciphertext, shift):
    decrypted_text = ""
    for char in ciphertext:
        if 'A' <= char <= 'Z':
            # Decode uppercase letters
            decrypted_char = chr(((ord(char) - ord('A') - shift) % 26) + ord('A'))
        elif 'a' <= char <= 'z':
            # Decode lowercase letters
            decrypted_char = chr(((ord(char) - ord('a') - shift) % 26) + ord('a'))
        else:
            # Leave non-alphabetical characters unchanged
            decrypted_char = char
        decrypted_text += decrypted_char
    return decrypted_text

# Example usage:
caesar_encoded_string = "ynkooejcpdanqxeykjrbdofgkq"
for i in range(26):
    shift = i
    decoded_string = decode_caesar_cipher(caesar_encoded_string, shift)
    print(decoded_string + " shift: " + str(shift))

print("Ans:" + decode_caesar_cipher(caesar_encoded_string, 22))


ynkooejcpdanqxeykjrbdofgkq shift: 0
xmjnndiboczmpwdxjiqacnefjp shift: 1
wlimmchanbylovcwihpzbmdeio shift: 2
vkhllbgzmaxknubvhgoyalcdhn shift: 3
ujgkkafylzwjmtaugfnxzkbcgm shift: 4
tifjjzexkyvilsztfemwyjabfl shift: 5
sheiiydwjxuhkrysedlvxizaek shift: 6
rgdhhxcviwtgjqxrdckuwhyzdj shift: 7
qfcggwbuhvsfipwqcbjtvgxyci shift: 8
pebffvatgurehovpbaisufwxbh shift: 9
odaeeuzsftqdgnuoazhrtevwag shift: 10
nczddtyrespcfmtnzygqsduvzf shift: 11
mbyccsxqdrobelsmyxfprctuye shift: 12
laxbbrwpcqnadkrlxweoqbstxd shift: 13
kzwaaqvobpmzcjqkwvdnparswc shift: 14
jyvzzpunaolybipjvucmozqrvb shift: 15
ixuyyotmznkxahoiutblnypqua shift: 16
hwtxxnslymjwzgnhtsakmxoptz shift: 17
gvswwmrkxlivyfmgsrzjlwnosy shift: 18
furvvlqjwkhuxelfrqyikvmnrx shift: 19
etquukpivjgtwdkeqpxhjulmqw shift: 20
dspttjohuifsvcjdpowgitklpv shift: 21
crossingtherubiconvfhsjkou shift: 22
bqnrrhmfsgdqtahbnmuegrijnt shift: 23
apmqqglerfcpszgamltdfqhims shift: 24
zolppfkdqeboryfzlkscepghlr shift: 25
Ans:crossingtherubiconvfhsjkou


Easy1

In [ ]:
# No code needed, solved using https://www.dcode.fr/vigenere-cipher with known ciphertext and key
# Can be solved manually looking at the table as well

Spelling-quiz

In [26]:
# Code obtained from writeup: https://www.ctfwriteup.com/picoctf/picomini-by-redpwn/cryptography

#!/usr/bin/env python3
from collections import Counter
from string import ascii_lowercase

with open('public/study-guide.txt', 'r') as f:
    ciphertext = f.read()

charcount = Counter(c for c in ciphertext if c in ascii_lowercase)
total_chars = sum(charcount.values())
for char, count in charcount.items():
    print(f"{char}: {(count*100)/total_chars}% ({count})")

# Next step is to use https://quipqiup.com/ with r=e as clue (obtained from freq. analysis)

g: 0.5909790436965906% (17173)
o: 3.6850415161659766% (107082)
c: 7.068519568760462% (205401)
n: 4.524140218923443% (131465)
f: 2.6330623403224385% (76513)
w: 9.29433530085455% (270080)
t: 7.46547660999031% (216936)
r: 10.715018225266496% (311363)
s: 2.9942639965641793% (87009)
x: 8.234544313276363% (239284)
l: 5.587028400581446% (162351)
y: 1.0493637675094705% (30493)
a: 7.101349826006519% (206355)
i: 7.391006299004493% (214772)
d: 2.286245429918069% (66435)
v: 6.820606389304907% (198197)
u: 1.7011166417055765% (49432)
b: 3.321878303673685% (96529)
q: 1.9856111245705224% (57699)
m: 3.1188056118403664% (90628)
p: 0.9449195004845388% (27458)
k: 0.40821017971984846% (11862)
e: 0.5141342172495815% (14940)
z: 0.28748843714210204% (8354)
j: 0.16497720465157256% (4794)
h: 0.11187753281649195% (3251)
